In [37]:
import pandas as pd

# Carrega o dataset já agrupado e com coluna 'Dia da Semana'
df = pd.read_excel("Itens_Agrupados_com_Dia_Semana.xlsx")
df.head()


,Data,Dia da Semana,Descrição,Qtde,Total
0,2024-03-12,Terça-feira,EXEC FRANGO A PARMEGIANA,6,227.4
1,2024-03-12,Terça-feira,EXEC MAMINHA AO MOLHO MADEIRA,4,151.6
2,2024-03-12,Terça-feira,EXEC TILAPIA AO POPEYE,7,272.3
3,2024-03-12,Terça-feira,MMT FRANGO PARMEGIANA,5,168.5
4,2024-03-12,Terça-feira,MMT LASANHA A BOLONHESA,5,156.5


Criação da coluna 'Vendido' (0 ou 1)

In [38]:
# Se a quantidade vendida for maior que 0, marcamos como 1 (foi vendido)
df["Vendido"] = (df["Qtde"] > 0).astype(int)


Pivotar os dados: linhas = datas | colunas = itens | valores = vendido (0/1)

In [39]:
# Transformar a tabela para ter 1 ou 0 por item em cada data
df_pivot = df.pivot_table(index="Data", columns="Descrição", values="Vendido", fill_value=0)

# Resetar índice pra facilitar
df_pivot = df_pivot.reset_index()


Criar colunas de features adicionais (dia da semana, mês)

In [40]:
df_pivot["Dia da Semana"] = pd.to_datetime(df_pivot["Data"]).dt.dayofweek  # 0=segunda
df_pivot["Mês"] = pd.to_datetime(df_pivot["Data"]).dt.month


Separar variáveis preditoras (X) e rótulos (Y)

In [41]:
# Features (dia da semana e mês)
X = df_pivot[["Dia da Semana", "Mês"]]

# Rótulos (colunas de itens)
Y = df_pivot.drop(columns=["Data", "Dia da Semana", "Mês"])


Divisão treino/teste (80/20)

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)


In [43]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# Remover colunas de Y_train com apenas uma classe (ex: só 0s)
Y_train = Y_train.loc[:, Y_train.nunique() > 1]
Y_test = Y_test[Y_train.columns]  # manter as mesmas colunas em X_test

# Treinar novamente
modelo = MultiOutputClassifier(LogisticRegression(max_iter=1000))
modelo.fit(X_train, Y_train)


MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000))

Treinamento com Regressão Logística Multi-label

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

modelo = MultiOutputClassifier(LogisticRegression(max_iter=1000))
modelo.fit(X_train, Y_train)


MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000))

Avaliação do modelo (Accuracy, Precision, Recall, F1)

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Y_pred = modelo.predict(X_test)

print("Acurácia:", accuracy_score(Y_test, Y_pred))
print("Precision:", precision_score(Y_test, Y_pred, average='macro'))
print("Recall:", recall_score(Y_test, Y_pred, average='macro'))
print("F1 Score:", f1_score(Y_test, Y_pred, average='macro'))


Acurácia: 0.0
Precision: 0.1506578947368421
Recall: 0.1303244670816827
F1 Score: 0.12941320805097895


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [46]:
# 4. Fazer previsões
Y_pred = modelo.predict(X_test)


In [47]:
# 5. Avaliação do modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Acurácia:", accuracy_score(Y_test, Y_pred))
print("Precision:", precision_score(Y_test, Y_pred, average="macro"))
print("Recall:", recall_score(Y_test, Y_pred, average="macro"))
print("F1 Score:", f1_score(Y_test, Y_pred, average="macro"))


Acurácia: 0.0
Precision: 0.1506578947368421
Recall: 0.1303244670816827
F1 Score: 0.12941320805097895


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [48]:
import json

In [49]:
acc = accuracy_score(Y_test, Y_pred)
prec = precision_score(Y_test, Y_pred, average="macro")
rec = recall_score(Y_test, Y_pred, average="macro")
f1 = f1_score(Y_test, Y_pred, average="macro")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [50]:
logistic_results = {
    "Modelo": "RL",
    "Tipo": "Classificação",
    "Acurácia": acc,
    "Precision": prec,
    "Recall": rec,
    "F1 Score": f1
}

In [51]:
with open("resultado_lstm.json", "r") as f:
    lstm_results = json.load(f)

print("LSTM carregado com sucesso!")


LSTM carregado com sucesso!


In [52]:

with open("resultado_regressao_linear.json", "r") as f:
    linear_results = json.load(f)

print("Regressão Linear carregada com sucesso!")


Regressão Linear carregada com sucesso!


In [53]:
comparacao_df = pd.DataFrame([logistic_results, lstm_results, linear_results])

print("\nComparação entre os modelos:")
display(comparacao_df)


Comparação entre os modelos:


,Modelo,Tipo,Acurácia,Precision,Recall,F1 Score,R² Score,MAE,RMSE
0,RL,Classificação,0.0,0.150658,0.130324,0.129413,NaN,NaN,NaN
1,LSTM,Classificação,0.0,0.392477,0.888683,0.512502,NaN,NaN,NaN
2,Regressão Linear,Regressão,NaN,NaN,NaN,NaN,-0.333346,1.564582,2.438587
